### Get public transport stop features

## Get publlic transport features from NDOV Loket and OpenStreetMap

In [ ]:
# Select where to run notebook: "azure" or "local"
my_run = "azure"

In [ ]:
import set_path

import pandas as pd
import xmltodict
import pyproj
import folium

import geopandas as gpd
import osmnx as ox

from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas()

import plot_utils
import poly_utils

import settings as st

if my_run == "azure":
    import config_azure as cf
elif my_run == "local":
    import config as cf

In [ ]:
os.system('sudo blobfuse /home/azureuser/cloudfiles/code/blobfuse/sidewalk --tmp-path=/mnt/resource/blobfusetmp --config-file=/home/azureuser/cloudfiles/code/blobfuse/fuse_connection_sidewalk.cfg -o attr_timeout=3600 -o entry_timeout=3600 -o negative_timeout=3600 -o allow_other -o nonempty')

#### Settings

In [ ]:
CRS = st.CRS
pd.set_option('display.max_columns', 500)

#### Import pilot area polygon

In [ ]:
df_areas = gpd.read_file(cf.output_pilot_area)
polygon = df_areas.to_crs(crs=CRS).unary_union

### 1. Obtain tram, bus and ferry stops including accessability information from NDOV Loket
source: https://data.ndovloket.nl/haltes/ \
help Jurian: https://github.com/jurb/NDOV-CHB-to-JSON/tree/master \
(make sure you pull the latest version and adjust the ndov_file variable in the subsequent cell accordingly) 

#### Create geodataframe containing bus and tram stops

In [ ]:
# Load xml with stop data
ndov_file = cf.ndov_public_transport_stops
with open(ndov_file) as xml_file:
    ndov_data = xmltodict.parse(xml_file.read(), process_namespaces=True, namespaces={"ns1":None, "http://bison.connekt.nl/tmi8/chb/msg": None})

In [ ]:
# Only keep stops in Amsterdam
adam_stops = [thing for thing in ndov_data["export"]["stopplaces"]["stopplace"] if thing["stopplacename"]["town"] == "Amsterdam"]

In [ ]:
# Configure xml stop data as dataframe
transformer = pyproj.Transformer.from_crs("EPSG:28992", "EPSG:4326")

quay_df = pd.DataFrame()

for stop in adam_stops:
    if stop["quays"] is None:
        continue
    quays = stop["quays"]["quay"] \
            if isinstance(stop["quays"]["quay"], list) \
            else [stop["quays"]["quay"]]

    for quay in quays:
        
        lat, lon = transformer.transform(float(quay["quaylocationdata"]["rd-x"]), float(quay["quaylocationdata"]["rd-y"]))
        quay_dict = {
            "stop_code": stop["stopplacecode"],
            "stop_type": stop["stopplacetype"],
            "stop_name": stop["stopplacename"]["publicname"],
            "stop_info_display": stop["stopplacefacilities"]["passengerinformationdisplay"],
            "type": quay["quaytypedata"]["quaytype"],
            "transportmodes": quay["quaytransportmodes"]["transportmodedata"]["transportmode"],
            "rd-x": quay["quaylocationdata"]["rd-x"],
            "rd-y": quay["quaylocationdata"]["rd-y"],
            "lat": lat,
            "lon": lon,
            "level": quay["quaylocationdata"]["level"],
            "bearing_compass_direction": quay["quaybearing"]["compassdirection"],
            "visually_accessible": quay["quayvisuallyaccessible"]["visuallyaccessible"],
            "visually_impaired_access": quay["quayvisuallyaccessible"]["visuallyImpairedAccess"],
            "disabled_access_mode": quay["quaydisabledaccessible"]["transportmode"],
            "disabled_access": quay["quaydisabledaccessible"]["disabledaccessible"],
            "disabled_access_step_free": quay["quaydisabledaccessible"]["stepFreeAccess"],
            "disabled_access_wheelchair": quay["quaydisabledaccessible"]["wheelchairAccess"],
            "remarks": quay["quayremarks"]["remarks"],
            
        }
        adaptions = quay.get("quayaccessibilityadaptions", {})
        adaptions.pop("validfrom", None)
        quay_dict.update({f"adapt-{key}": val for key, val in adaptions.items()})

        facilities = quay.get("quayfacilities", {})
        facilities.pop("validfrom", None)
        quay_dict.update({f"facil-{key}": val for key, val in facilities.items()})

        extras = quay.get("quayextraattributes", {})
        extras.pop("validfrom", None)
        quay_dict.update({f"facil-{key}": val for key, val in extras.items()})

        quay_df = pd.concat([quay_df, pd.DataFrame([quay_dict])], ignore_index=True)

In [ ]:
# Create geodataframe from stop information dataframe
quay_gdf = gpd.GeoDataFrame(
    quay_df, geometry=gpd.points_from_xy(quay_df.lon, quay_df.lat), crs="EPSG:4326"
).to_crs(st.CRS)

In [ ]:
# Only keep stops within pilot area
quay_gdf_mask = polygon.contains(quay_gdf['geometry'])
all_features = quay_gdf.loc[quay_gdf_mask]
all_features = all_features.rename(columns={'disabled_access': 'wheelchair_accessible', 'type': 'stop_placement_type', 'adapt-narrowestpassagewidth': 'obstacle_free_width_float'})
all_features['obstacle_free_width_float'] = all_features['obstacle_free_width_float'].astype(float)

In [ ]:
# Rename columns and change variable names
all_features['wheelchair_accessible'] = all_features['wheelchair_accessible'].str.replace('Y', 'Yes')
all_features['wheelchair_accessible'] = all_features['wheelchair_accessible'].str.replace('N', 'No')
all_features['wheelchair_accessible'] = all_features['wheelchair_accessible'].str.replace('U', 'Unknown')

In [ ]:
# Get features seperately
bus_features = all_features.loc[all_features['stop_type'] == 'onstreetBus']
tram_features = all_features.loc[all_features['stop_type'] == 'onstreetTram']
ferry_features = all_features.loc[all_features['stop_type'] == 'ferryPort']
bus_tram_features = all_features.loc[all_features['stop_type'] == 'combiTramBus']
metro_features_osm = all_features.loc[all_features['stop_type'] == 'metroStation']
other_features = all_features.loc[~all_features['stop_type'].isin(['onstreetBus', 'onstreetTram', 'ferryPort', 'combiTramBus', 'metroStation'])]

### 2. Obtain metro stops from OpenStreetMap


In [ ]:
# Retrieve crossing attributes
CRS_metro = 'EPSG:4326'
polygon_metro = df_areas.to_crs(crs=CRS_metro).unary_union
tags = {"railway": 'subway_entrance'}
metro_features = ox.features_from_polygon(polygon_metro, tags)
metro_features = metro_features.to_crs(crs=CRS)
metro_features = metro_features.rename(columns={'name': 'stop_name', 'layer': 'level', 'wheelchair': 'wheelchair_accessible'})
metro_features['stop_type'] = 'metroStation'
metro_features['stop_placement_type'] = 'regular'

metro_features['wheelchair_accessible'].fillna('Unkown', inplace=True)
metro_features['wheelchair_accessible'] = metro_features['wheelchair_accessible'].str.capitalize()

# Impute missing stop_name values
metro_features = poly_utils.infer_column_by_distance(metro_features, metro_features_osm, 'stop_name')
metro_features = metro_features.loc[~metro_features.level.isin(['-1', '-2', '-3'])]
open_idxs = [i for i, row in metro_features.iterrows() if not 'closed' in str(row['stop_name']).lower()]
metro_features = metro_features.loc[open_idxs]

### 3. Visualize stops

In [ ]:
# set True for satelite background, False for regular background
satelite = False

# Create Folium map
map = folium.Map(
    location=[52.389164, 4.908453], tiles=plot_utils.generate_map_params(satelite=satelite),
    min_zoom=10, max_zoom=25, zoom_start=15,
    zoom_control=True, control_scale=True, control=False
    )

# Add transport features
feature_names = all_features.columns.tolist()
feature_names.remove('geometry')
color_column = 'wheelchair_accessible'
geo_j = folium.GeoJson(bus_features, tooltip=plot_utils.gen_tooltip(feature_names, feature_names), marker=folium.Marker(icon=folium.Icon(icon='solid fa-bus', prefix='fa')), style_function =lambda feature: {"markerColor": 'green' if feature["properties"][color_column] == 'Yes' else 
                                                                                                                                              'red' if feature["properties"][color_column] == 'No' else
                                                                                                                                              'orange'}).add_to(map)
geo_j = folium.GeoJson(tram_features, tooltip=plot_utils.gen_tooltip(feature_names, feature_names), marker=folium.Marker(icon=folium.Icon(icon='solid fa-train-tram', prefix='fa')), style_function =lambda feature: {"markerColor": 'green' if feature["properties"][color_column] == 'Yes' else 
                                                                                                                                              'red' if feature["properties"][color_column] == 'No' else
                                                                                                                                              'orange'}).add_to(map)

geo_j = folium.GeoJson(bus_tram_features, tooltip=plot_utils.gen_tooltip(feature_names, feature_names), marker=folium.Marker(icon=folium.Icon(icon='solid fa-2', prefix='fa')), style_function =lambda feature: {"markerColor": 'green' if feature["properties"][color_column] == 'Yes' else 
                                                                                                                                              'red' if feature["properties"][color_column] == 'No' else
                                                                                                                                              'orange'}).add_to(map)

geo_j = folium.GeoJson(ferry_features, tooltip=plot_utils.gen_tooltip(feature_names, feature_names), marker=folium.Marker(icon=folium.Icon(icon='solid fa-ferry', prefix='fa')), style_function =lambda feature: {"markerColor": 'green' if feature["properties"][color_column] == 'Yes' else 
                                                                                                                                              'red' if feature["properties"][color_column] == 'No' else
                                                                                                                                              'orange'}).add_to(map)                                                                                                                                      
# geo_j = folium.GeoJson(other_features, tooltip=plot_utils.gen_tooltip(feature_names, feature_names), marker=folium.Marker(icon=folium.Icon(icon='solid fa-question', prefix='fa')), style_function =lambda feature: {"markerColor": 'green' if feature["properties"][color_column] == 'Y' else 
#                                                                                                                                               'red' if feature["properties"][color_column] == 'N' else
#                                                                                                                                               'orange'}).add_to(map)

feature_names = metro_features.columns.tolist()
feature_names.remove('geometry')
tooltip = plot_utils.gen_tooltip(feature_names, feature_names) 
geo_j = folium.GeoJson(metro_features, tooltip=plot_utils.gen_tooltip(feature_names, feature_names), marker=folium.Marker(icon=folium.Icon(icon='solid fa-train', prefix='fa')), style_function =lambda feature: {"markerColor": 'green' if feature["properties"][color_column] == 'Yes' else 
                                                                                                                                              'red' if feature["properties"][color_column] == 'No' else
                                                                                                                                              'orange'}).add_to(map)
                                   
map

### 4. Save features as geopackage

In [ ]:
public_transport_features = pd.concat([bus_features, tram_features, bus_tram_features, ferry_features, metro_features])
public_transport_features.reset_index(drop=True, inplace=True)
public_transport_features = public_transport_features.drop(columns=public_transport_features.columns.difference(['geometry', 'stop_type', 'stop_name', 'wheelchair_accessible', 'stop_placement_type', 'obstacle_free_width_float']))
public_transport_features.to_file(cf.output_public_transport_features, driver='GPKG')